<h2 align="center">Собираем Датасет на основе музыки Spotify</h2> 

In [2]:
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import librosa as lr
import pydub
import math
from sklearn.decomposition import PCA



In [9]:
path = 'Path to Spotify'
genres = os.listdir(path)
del genres[1]
path2 = 'Path to Y.Music'
genresgt = os.listdir(path2)
del genresgt[1]


Посмотрим сколько песен мы можем загрузить по длительности и количеству(возможно потом это будет важно)

In [4]:
msmin_bygenre = dict(list(zip(genres,[0 for _ in range(10)])))
numsong = list()
for genre in genres:
    songs = os.listdir(path+f'/{genre}')
    numsong.append(len(songs))
minsong = min(numsong)
numsong

[46, 43, 40, 45, 48, 48, 46, 50, 46, 47]

In [4]:
msmin_bygenregt = dict(list(zip(genresgt,[0 for _ in range(10)])))
for genre in genresgt:
    songs = os.listdir(path2+f'/{genre}')
    dlit = list()
    for j,s in tqdm(enumerate(songs)):
        try:
            dur = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration']
            data, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        except:
            continue
        dlit.append(data.shape[0])
    msmin_bygenregt[genre] = min(dlit)

100it [00:15,  6.39it/s]
100it [00:13,  7.19it/s]
100it [00:14,  6.74it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.95it/s]
100it [00:14,  6.93it/s]
100it [00:14,  7.13it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.76it/s]


In [636]:
msmin_bygenregt

{'pop': 661500,
 'metal': 661500,
 'disco': 639450,
 'blues': 661500,
 'reggae': 661500,
 'classical': 639450,
 'rock': 639450,
 'hiphop': 639450,
 'country': 639450,
 'jazz': 661500}

#### Загружаем песни из датасета GTZAN
**GTZAN**
* Создан в ~2000 году
* Состоит из 10 жанров по 100 песен, длительностью 30 секунд
* Песня начинается с произвольного момента(однозначно не начало)
* При прослушивании некоторых песен что-то режет слух; Возможно в песнях есть какой-то шум, битый звук
* Непонятно, как собирались данные, нет кода по сбору\

Из всех предпосылок выше был сделан вывод, что этот датасет не стоит использовать. Однако код по сбору признаков остался.

In [396]:
gtzan_genre = dict(list(zip(genresgt,[0 for _ in range(10)])))
for genre in genresgt:
    ans = np.zeros((55,))
    songs = os.listdir(path2+f'/{genre}')
    print(genre)
    for j,s in tqdm(enumerate(songs)):
        if s == '.DS_Store':
            pass
        durts = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration_ts']
        datak, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(durts)))
        for i in range(10):
            data = datak[i*66149:66149*(i+1)]
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            prom = list()
            for feat in [mfccmean,mfccvar,stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0]]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
    gtzan_genre[genre] = ans

pop


100it [03:26,  2.06s/it]


metal


100it [03:28,  2.08s/it]


disco


...

country


100it [03:54,  2.34s/it]


jazz


44it [01:38,  2.24s/it]


KeyError: 'duration_ts'

Ошибка, продолжим собирать, используя костыли

In [433]:
ans = np.zeros((55,))
songs = os.listdir(path2+f'/{genre}')
print(genre)
for j,s in tqdm(enumerate(songs)):
    if s == '.DS_Store' or s =='jazz.00054.wav':
        continue
    durts = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration_ts']
    datak, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(durts)))
    for i in range(10):
        data = datak[i*66149:66149*(i+1)]
        mfcc = lr.feature.mfcc(y=data,sr=sr)
        stft = lr.feature.chroma_stft(y=data, sr=sr)
        rms = lr.feature.rms(y=data)
        centroid = lr.feature.spectral_centroid(y=data, sr=sr)
        bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
        rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
        zero = lr.feature.zero_crossing_rate(y=data)
        harm = lr.effects.harmonic(y=data)
        temp = lr.feature.tempo(y=data)
        mfccmean = np.mean(mfcc,axis=1)
        mfccvar = np.std(mfcc,axis=1)
        stftmean = np.mean(stft)
        stftvar = np.std(stft)
        rmsmean = np.mean(rms)
        rmsvar = np.std(rms)
        centroidmean = np.mean(centroid)
        centroidvar = np.std(centroid)
        bandwithmean = np.mean(bandwith)
        bandwithvar = np.std(bandwith)
        rolloffmean = np.mean(rolloff)
        rolloffvar = np.std(rolloff)
        zeromean = np.mean(zero)
        zerovar = np.std(zero)
        harmmean = np.mean(harm)
        harmvar = np.std(harm)
        prom = list()
        for feat in [mfccmean,mfccvar,stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0]]:
            if type(feat) == np.ndarray:
                prom += feat.tolist()
            else:
                prom.append(feat)
        prom = np.array(prom)
        ans = np.vstack((prom,ans))
gtzan_genre[genre] = ans

jazz


100it [03:32,  2.12s/it]


#### Датасет из песен Spotify
**Каким образом?** \
Мы используем несколько признаков из библиотеки `librosa.feature`, таких как: MFCC, Chroma STFT, RMS, Spectral bandwith, centroid, rolloff, zero crossing rate, harmonic, tempo. Подробнее об этих методах библиотеки librosa описано в **librosa_preview.ipynb**. Все эти методы на вход принимают предобработанный аудиофайл - временной ряд, который загружается через `librosa.feature.load`. Так как некоторые из этих методов на выходе выдают матрицы(MFCC), а некоторые выдают np.array, то принято решение брать среднее и стандартное отклонение. Для матриц: mean и std по строкам, для векторов: mean и std по единственной строке. Это сделано для того, чтобы во-первых, уменьшить количество признаков, дабы избежать проблемы переобучения(многие из этих методов могут выдавать векторы размерностью до 1000 или более), а во-вторых, чтобы агрегировать данные.\
**Агрегация данных**\
Использование среднего и стандартного отклонения агрегирует информацию, что плохо для нашего случая. Почему? Каждый из этих методов обрабатывает временной ряд так, чтобы на выходе выдать информацию, которая представляет собой либо матрицу, либо вектор. В случае с вектором, каждое его значение будет показывать определенную "черту"(в зависимости от метода) трека в определенный момент времени. Поэтому агрегировать данные по времени не совсем хорошо - мы теряем часть информации и по итогу говорим лишь о том, как в целом выглядит эта "черта" за все время.\
**Борьба с агрегацией** \
Пусть у нас есть трек длительностью, например, 1.5 минуты(90 секунд). Если агрегировать информацию по всему треку сразу, то мы потеряем слишком много информации и собственно, датасет будет содержать в себе так мало данных, что обучиться на них хорошо будет нереально. Поэтому разобьем трек на отрезки по 3 секунды и будем по ним считать librosa.feature и потом по ним среднее и стандартное отклонение. Это позволит агрегировать информацию на меньшем отрезке и терять меньше данных.


In [16]:
genresgt

['pop',
 'metal',
 'disco',
 'blues',
 'reggae',
 'classical',
 'rock',
 'hiphop',
 'country',
 'jazz']

In [34]:
import warnings
warnings.filterwarnings('ignore')
spotify_genre = dict(list(zip(genres,[0 for _ in range(10)])))
for k,genre in enumerate(genresgt):
    genre = genre+'_spotify'
    if genre == 'hiphop_spotify':
        genre = 'hip hop_spotify'
    ans = np.zeros((311,))
    songs = os.listdir(path+f'/{genre}')
    print(genre)
    for j,s in (enumerate(songs)):
        if j == 39:
            break
        if s == '.DS_Store':
            continue
        try:
            dur = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration']
            durts = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration_ts']
        except:
            pass
        datak, sr = lr.load(path+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        count = math.floor(float(dur)/3)
        i = 0
        print(count)
        while i <= count:
            data = datak[i*66149:66149*(i+1)]
            mel = lr.feature.melspectrogram(y=data,sr=sr)
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            melmean = np.mean(mel,axis=1)
            melvar = np.std(mel,axis=1)
            prom = list()
            for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar,melmean,melvar]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
            i+=1
    ans = np.hstack((ans,np.array([k for i in range(ans.shape[0])]).reshape(-1,1)))
    spotify_genre[genre] = ans

pop_spotify
85
9
41
67
70
73
76
61
72
66
60
40
66
73
77
71
78
76
69
67
69
69
58
66
82
9
66
82
58
70
85
83
75
68
75
61
78
73
69
metal_spotify
80
96
105
63
81
85
126
88
81
70
58
84
95
80
113
138
67
104
115
86
74
93
84
90
109
97
127
140
71
86
96
131
94
71
79
103
105
129
113
disco_spotify
78
78
63
74
82
61
81
95
71
68
95
143
82
131
125
87
98
60
80
80
77
58
82
75
68
77
63
111
71
57
67
71
100
95
61
72
83
113
93
blues_spotify
137
48
56
67
170
78
79
91
115
46
49
87
54
63
57
106
172
63
56
86
39
56
60
57
59
55
59
63
62
56
49
72
55
43
56
75
57
48
95
reggae_spotify
94
105
79
85
72
74
72
77
71
54
57
98
91
80
76
72
59
69
103
79
88
85
75
50
77
156
77
73
59
92
82
116
59
71
92
67
96
82
65
classical_spotify
85
76
60
19
104
86
64
89
56
164
100
43
89
29
115
37
74
238
75
56
59
50
21
53
57
92
82
39
59
119
250
142
47
79
69
45
81
84
rock_spotify
72
89
76
55
90
62
74
84
88
168
70
66
118
93
54
52
76
90
69
85
59
109
48
135
88
103
74
59
94
83
74
111
77
110
66
66
90
172
68
hip hop_spotify
95
96
83
53
115
99
91
97


Собрали данные по жанрам, загрузим их в DataFrame

In [36]:
for genre in spotify_genre:
    spotify_genre[genre] = spotify_genre[genre][:-1]
df = pd.DataFrame(np.vstack((spotify_genre.values())))
df

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.243564,0.302197,0.000259,0.000331,2379.867100,1583.673987,1988.863977,1177.816996,4596.225081,3070.398615,...,0.000003,0.000002,0.000002,0.000001,0.000001,9.547306e-07,7.960268e-07,3.732888e-07,3.126712e-08,4.0
1,0.398592,0.290751,0.008298,0.006522,2549.703359,902.814666,2434.436152,621.608576,5227.102239,2044.925894,...,0.001251,0.000947,0.000886,0.001016,0.000698,7.218310e-04,1.213449e-03,7.527992e-04,5.277236e-05,4.0
2,0.332872,0.306482,0.045383,0.020386,2496.088753,1065.052254,2415.676166,735.048307,5054.339694,2311.583265,...,0.082897,0.042400,0.045215,0.046453,0.042490,4.016979e-02,4.328870e-02,2.639336e-02,2.995682e-03,4.0
3,0.374518,0.291596,0.106776,0.052839,2804.832168,1061.562911,2544.598453,567.635555,5698.016827,1970.422738,...,0.182320,0.119765,0.088639,0.094064,0.069251,5.209206e-02,6.751129e-02,4.142033e-02,3.045732e-03,4.0
4,0.376507,0.290995,0.158438,0.060707,2622.996230,1036.870399,2500.696752,622.321587,5380.319261,2131.597207,...,0.384795,0.200407,0.171049,0.307872,0.199622,1.875423e-01,2.183229e-01,1.948708e-01,1.540417e-02,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,0.243937,0.301781,0.139670,0.054711,1439.718324,299.060633,1772.859100,377.912478,2788.963905,1052.133075,...,0.002715,0.002352,0.002370,0.001726,0.001804,1.724900e-03,1.624034e-03,4.577331e-04,9.622837e-05,9.0
32129,0.321663,0.313815,0.128225,0.062764,1396.735413,338.528844,1877.858331,372.184717,2867.145996,1089.823969,...,0.003103,0.002224,0.002239,0.002143,0.001602,1.320671e-03,1.335345e-03,5.398928e-04,9.476399e-05,9.0
32130,0.376502,0.301102,0.093401,0.033267,1296.653143,408.401490,1854.195611,419.549387,2639.556603,1283.754184,...,0.003476,0.002583,0.002012,0.002016,0.001980,1.512903e-03,1.070709e-03,4.319110e-04,5.009029e-05,9.0
32131,0.312806,0.306806,0.125220,0.053452,1361.608808,367.246603,1845.474956,403.255781,2781.758564,1113.840538,...,0.002933,0.002200,0.001096,0.001114,0.001048,9.549149e-04,1.172733e-03,6.207458e-04,4.291250e-05,9.0


In [37]:
df.to_csv('spotify.csv')

На данный момент загрузим "по тупому" train и test классы, дабы избежать лика в данных.

In [671]:
class_train_1 = df.iloc[:1789,:]
class_test_1 = df.iloc[1789:2347,:]
class_train_2 = df.iloc[2347:4651,:]
class_test_2 = df.iloc[4651:4651+988,:]
class_train_3 = df.iloc[5639:7428,:]
class_test_3 = df.iloc[7428:8539,:]
class_train_4 = df.iloc[8539:10328,:]
class_test_4 = df.iloc[10328:11095,:]
class_train_5 = df.iloc[11095:13093,:]
class_test_5 = df.iloc[13093:13949,:]
class_train_6 = df.iloc[13949:15938,:]
class_test_6 = df.iloc[15938:16791,:]
class_train_7 = df.iloc[16791:18860,:]
class_test_7 = df.iloc[18860:19747,:]
class_train_8 = df.iloc[19747:21837,:]
class_test_8 = df.iloc[21837:22733,:]
class_train_9 = df.iloc[22733:24823,:]
class_test_9 = df.iloc[24823:25236,:]
class_train_10 = df.iloc[25236:27726,:]
class_test_10 = df.iloc[27726:28793,:]

In [672]:
test = pd.DataFrame(np.vstack((class_test_1,class_test_2,class_test_3,class_test_4,
                                class_test_5,class_test_6,class_test_7,class_test_8,class_test_9,class_test_10)))
train = pd.DataFrame(np.vstack((class_train_1,class_train_2,class_train_3,
                                 class_train_4,class_test_5,class_train_6,class_train_7,
                                 class_train_8,class_train_9,class_train_10)))
X_train = train.iloc[:,:-1]
y_train = train[55]
X_test = test.iloc[:,:-1]
y_test = test[55]

Сохраним датасет в формате .csv

In [673]:
df.to_csv('spotify.csv')

In [60]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
clf = CatBoostClassifier()
X_train,X_test,y_train,y_test = train_test_split(df2.iloc[:,:-1],df2.iloc[:,-1],test_size=0.3)

In [61]:
clf.fit(X_train,y_train)

Learning rate set to 0.091173
0:	learn: 2.1601119	total: 264ms	remaining: 4m 23s
1:	learn: 2.0615368	total: 551ms	remaining: 4m 35s
2:	learn: 1.9809606	total: 808ms	remaining: 4m 28s
3:	learn: 1.9152003	total: 1.07s	remaining: 4m 25s
4:	learn: 1.8556189	total: 1.37s	remaining: 4m 31s
5:	learn: 1.8001827	total: 1.63s	remaining: 4m 30s
6:	learn: 1.7585381	total: 1.9s	remaining: 4m 29s
7:	learn: 1.7199287	total: 2.18s	remaining: 4m 29s
8:	learn: 1.6818635	total: 2.49s	remaining: 4m 34s
9:	learn: 1.6485032	total: 2.82s	remaining: 4m 39s
10:	learn: 1.6149035	total: 3.12s	remaining: 4m 40s
11:	learn: 1.5815771	total: 3.39s	remaining: 4m 39s
12:	learn: 1.5495722	total: 3.66s	remaining: 4m 37s
13:	learn: 1.5216860	total: 4.02s	remaining: 4m 42s
14:	learn: 1.4987041	total: 4.36s	remaining: 4m 46s
15:	learn: 1.4762589	total: 4.62s	remaining: 4m 44s
16:	learn: 1.4555853	total: 4.89s	remaining: 4m 42s
17:	learn: 1.4378965	total: 5.15s	remaining: 4m 41s
18:	learn: 1.4188086	total: 5.42s	remaining: 

157:	learn: 0.6465593	total: 46.6s	remaining: 4m 8s
158:	learn: 0.6443757	total: 47s	remaining: 4m 8s
159:	learn: 0.6419435	total: 47.2s	remaining: 4m 8s
160:	learn: 0.6391655	total: 47.5s	remaining: 4m 7s
161:	learn: 0.6370028	total: 47.8s	remaining: 4m 7s
162:	learn: 0.6347094	total: 48.1s	remaining: 4m 7s
163:	learn: 0.6317573	total: 48.4s	remaining: 4m 6s
164:	learn: 0.6292129	total: 48.7s	remaining: 4m 6s
165:	learn: 0.6269040	total: 49s	remaining: 4m 6s
166:	learn: 0.6240152	total: 49.4s	remaining: 4m 6s
167:	learn: 0.6213253	total: 49.8s	remaining: 4m 6s
168:	learn: 0.6194169	total: 50.1s	remaining: 4m 6s
169:	learn: 0.6169532	total: 50.5s	remaining: 4m 6s
170:	learn: 0.6142824	total: 50.9s	remaining: 4m 6s
171:	learn: 0.6117841	total: 51.2s	remaining: 4m 6s
172:	learn: 0.6089303	total: 51.6s	remaining: 4m 6s
173:	learn: 0.6061176	total: 52s	remaining: 4m 6s
174:	learn: 0.6040824	total: 52.3s	remaining: 4m 6s
175:	learn: 0.6024938	total: 52.6s	remaining: 4m 6s
176:	learn: 0.6004

313:	learn: 0.4126956	total: 1m 38s	remaining: 3m 35s
314:	learn: 0.4120063	total: 1m 38s	remaining: 3m 34s
315:	learn: 0.4115700	total: 1m 39s	remaining: 3m 34s
316:	learn: 0.4112319	total: 1m 39s	remaining: 3m 34s
317:	learn: 0.4103091	total: 1m 39s	remaining: 3m 33s
318:	learn: 0.4091778	total: 1m 40s	remaining: 3m 33s
319:	learn: 0.4086064	total: 1m 40s	remaining: 3m 33s
320:	learn: 0.4079688	total: 1m 40s	remaining: 3m 32s
321:	learn: 0.4069600	total: 1m 40s	remaining: 3m 32s
322:	learn: 0.4058638	total: 1m 41s	remaining: 3m 32s
323:	learn: 0.4052354	total: 1m 41s	remaining: 3m 31s
324:	learn: 0.4032804	total: 1m 41s	remaining: 3m 31s
325:	learn: 0.4023737	total: 1m 42s	remaining: 3m 31s
326:	learn: 0.4018664	total: 1m 42s	remaining: 3m 30s
327:	learn: 0.4006270	total: 1m 42s	remaining: 3m 30s
328:	learn: 0.3995538	total: 1m 43s	remaining: 3m 30s
329:	learn: 0.3983565	total: 1m 43s	remaining: 3m 29s
330:	learn: 0.3979718	total: 1m 43s	remaining: 3m 29s
331:	learn: 0.3975137	total:

467:	learn: 0.3092520	total: 2m 30s	remaining: 2m 51s
468:	learn: 0.3090170	total: 2m 30s	remaining: 2m 50s
469:	learn: 0.3082374	total: 2m 31s	remaining: 2m 50s
470:	learn: 0.3075628	total: 2m 31s	remaining: 2m 50s
471:	learn: 0.3070963	total: 2m 32s	remaining: 2m 50s
472:	learn: 0.3068413	total: 2m 32s	remaining: 2m 49s
473:	learn: 0.3065863	total: 2m 32s	remaining: 2m 49s
474:	learn: 0.3062240	total: 2m 33s	remaining: 2m 49s
475:	learn: 0.3059098	total: 2m 33s	remaining: 2m 48s
476:	learn: 0.3051993	total: 2m 33s	remaining: 2m 48s
477:	learn: 0.3048142	total: 2m 34s	remaining: 2m 48s
478:	learn: 0.3044267	total: 2m 34s	remaining: 2m 47s
479:	learn: 0.3041638	total: 2m 34s	remaining: 2m 47s
480:	learn: 0.3036896	total: 2m 35s	remaining: 2m 47s
481:	learn: 0.3034648	total: 2m 35s	remaining: 2m 47s
482:	learn: 0.3029812	total: 2m 35s	remaining: 2m 46s
483:	learn: 0.3021986	total: 2m 36s	remaining: 2m 46s
484:	learn: 0.3013309	total: 2m 36s	remaining: 2m 46s
485:	learn: 0.3004163	total:

620:	learn: 0.2462057	total: 3m 22s	remaining: 2m 3s
621:	learn: 0.2460264	total: 3m 22s	remaining: 2m 3s
622:	learn: 0.2454392	total: 3m 22s	remaining: 2m 2s
623:	learn: 0.2451783	total: 3m 23s	remaining: 2m 2s
624:	learn: 0.2448481	total: 3m 23s	remaining: 2m 2s
625:	learn: 0.2444776	total: 3m 23s	remaining: 2m 1s
626:	learn: 0.2440730	total: 3m 24s	remaining: 2m 1s
627:	learn: 0.2439731	total: 3m 24s	remaining: 2m 1s
628:	learn: 0.2433567	total: 3m 24s	remaining: 2m
629:	learn: 0.2429426	total: 3m 25s	remaining: 2m
630:	learn: 0.2426597	total: 3m 25s	remaining: 2m
631:	learn: 0.2425825	total: 3m 25s	remaining: 1m 59s
632:	learn: 0.2424001	total: 3m 25s	remaining: 1m 59s
633:	learn: 0.2421963	total: 3m 26s	remaining: 1m 59s
634:	learn: 0.2419362	total: 3m 26s	remaining: 1m 58s
635:	learn: 0.2415720	total: 3m 26s	remaining: 1m 58s
636:	learn: 0.2410013	total: 3m 27s	remaining: 1m 58s
637:	learn: 0.2407577	total: 3m 27s	remaining: 1m 57s
638:	learn: 0.2406588	total: 3m 27s	remaining: 1

773:	learn: 0.2031936	total: 4m 12s	remaining: 1m 13s
774:	learn: 0.2029294	total: 4m 13s	remaining: 1m 13s
775:	learn: 0.2027587	total: 4m 13s	remaining: 1m 13s
776:	learn: 0.2025570	total: 4m 13s	remaining: 1m 12s
777:	learn: 0.2022320	total: 4m 14s	remaining: 1m 12s
778:	learn: 0.2019829	total: 4m 14s	remaining: 1m 12s
779:	learn: 0.2017941	total: 4m 14s	remaining: 1m 11s
780:	learn: 0.2016738	total: 4m 15s	remaining: 1m 11s
781:	learn: 0.2012513	total: 4m 15s	remaining: 1m 11s
782:	learn: 0.2011306	total: 4m 15s	remaining: 1m 10s
783:	learn: 0.2009971	total: 4m 16s	remaining: 1m 10s
784:	learn: 0.2007718	total: 4m 16s	remaining: 1m 10s
785:	learn: 0.2006073	total: 4m 16s	remaining: 1m 9s
786:	learn: 0.2000413	total: 4m 17s	remaining: 1m 9s
787:	learn: 0.1998206	total: 4m 17s	remaining: 1m 9s
788:	learn: 0.1997197	total: 4m 17s	remaining: 1m 8s
789:	learn: 0.1995726	total: 4m 17s	remaining: 1m 8s
790:	learn: 0.1993587	total: 4m 18s	remaining: 1m 8s
791:	learn: 0.1991574	total: 4m 18

929:	learn: 0.1708357	total: 5m 5s	remaining: 23s
930:	learn: 0.1706644	total: 5m 5s	remaining: 22.6s
931:	learn: 0.1705536	total: 5m 5s	remaining: 22.3s
932:	learn: 0.1703234	total: 5m 6s	remaining: 22s
933:	learn: 0.1702309	total: 5m 6s	remaining: 21.7s
934:	learn: 0.1701634	total: 5m 6s	remaining: 21.3s
935:	learn: 0.1699610	total: 5m 7s	remaining: 21s
936:	learn: 0.1697165	total: 5m 7s	remaining: 20.7s
937:	learn: 0.1696117	total: 5m 7s	remaining: 20.3s
938:	learn: 0.1695241	total: 5m 8s	remaining: 20s
939:	learn: 0.1693417	total: 5m 8s	remaining: 19.7s
940:	learn: 0.1692267	total: 5m 8s	remaining: 19.4s
941:	learn: 0.1691337	total: 5m 8s	remaining: 19s
942:	learn: 0.1690804	total: 5m 9s	remaining: 18.7s
943:	learn: 0.1689046	total: 5m 9s	remaining: 18.4s
944:	learn: 0.1688198	total: 5m 9s	remaining: 18s
945:	learn: 0.1687080	total: 5m 10s	remaining: 17.7s
946:	learn: 0.1685902	total: 5m 10s	remaining: 17.4s
947:	learn: 0.1684109	total: 5m 10s	remaining: 17s
948:	learn: 0.1682740	t

Проверим на незнакомых данных

In [72]:
df3 = pd.DataFrame(np.vstack((np.array(df),np.array(df2))))
X_train,X_test,y_train,y_test = train_test_split(df3.iloc[:,:-1],df3.iloc[:,-1],test_size=0.3)
clf.fit(X_train,y_train)

Learning rate set to 0.095331
0:	learn: 2.1488017	total: 474ms	remaining: 7m 53s
1:	learn: 2.0511203	total: 932ms	remaining: 7m 45s
2:	learn: 1.9697475	total: 1.38s	remaining: 7m 38s
3:	learn: 1.9096378	total: 1.85s	remaining: 7m 41s
4:	learn: 1.8497080	total: 2.3s	remaining: 7m 37s
5:	learn: 1.7975946	total: 2.75s	remaining: 7m 35s
6:	learn: 1.7498380	total: 3.19s	remaining: 7m 33s
7:	learn: 1.7082666	total: 3.64s	remaining: 7m 31s
8:	learn: 1.6742906	total: 4.11s	remaining: 7m 32s
9:	learn: 1.6423252	total: 4.55s	remaining: 7m 30s
10:	learn: 1.6147140	total: 5s	remaining: 7m 29s
11:	learn: 1.5872721	total: 5.44s	remaining: 7m 28s
12:	learn: 1.5622749	total: 5.9s	remaining: 7m 28s
13:	learn: 1.5377442	total: 6.37s	remaining: 7m 28s
14:	learn: 1.5157539	total: 6.84s	remaining: 7m 29s
15:	learn: 1.4946834	total: 7.31s	remaining: 7m 29s
16:	learn: 1.4745462	total: 7.79s	remaining: 7m 30s
17:	learn: 1.4545971	total: 8.31s	remaining: 7m 33s
18:	learn: 1.4360244	total: 8.85s	remaining: 7m 3

157:	learn: 0.7617084	total: 1m 23s	remaining: 7m 23s
158:	learn: 0.7604193	total: 1m 23s	remaining: 7m 22s
159:	learn: 0.7587215	total: 1m 24s	remaining: 7m 22s
160:	learn: 0.7566270	total: 1m 24s	remaining: 7m 22s
161:	learn: 0.7545969	total: 1m 25s	remaining: 7m 21s
162:	learn: 0.7527379	total: 1m 25s	remaining: 7m 21s
163:	learn: 0.7503683	total: 1m 26s	remaining: 7m 20s
164:	learn: 0.7484460	total: 1m 26s	remaining: 7m 19s
165:	learn: 0.7463391	total: 1m 27s	remaining: 7m 19s
166:	learn: 0.7441980	total: 1m 28s	remaining: 7m 19s
167:	learn: 0.7428293	total: 1m 28s	remaining: 7m 18s
168:	learn: 0.7400924	total: 1m 29s	remaining: 7m 17s
169:	learn: 0.7382181	total: 1m 29s	remaining: 7m 17s
170:	learn: 0.7368865	total: 1m 30s	remaining: 7m 16s
171:	learn: 0.7352964	total: 1m 30s	remaining: 7m 16s
172:	learn: 0.7323964	total: 1m 31s	remaining: 7m 15s
173:	learn: 0.7309647	total: 1m 31s	remaining: 7m 15s
174:	learn: 0.7284999	total: 1m 32s	remaining: 7m 15s
175:	learn: 0.7262346	total:

310:	learn: 0.5440269	total: 2m 46s	remaining: 6m 8s
311:	learn: 0.5426743	total: 2m 47s	remaining: 6m 8s
312:	learn: 0.5416932	total: 2m 47s	remaining: 6m 7s
313:	learn: 0.5405808	total: 2m 48s	remaining: 6m 7s
314:	learn: 0.5395568	total: 2m 48s	remaining: 6m 6s
315:	learn: 0.5384915	total: 2m 49s	remaining: 6m 6s
316:	learn: 0.5376945	total: 2m 49s	remaining: 6m 5s
317:	learn: 0.5364505	total: 2m 50s	remaining: 6m 5s
318:	learn: 0.5357594	total: 2m 50s	remaining: 6m 4s
319:	learn: 0.5343379	total: 2m 51s	remaining: 6m 3s
320:	learn: 0.5334985	total: 2m 51s	remaining: 6m 3s
321:	learn: 0.5327763	total: 2m 52s	remaining: 6m 2s
322:	learn: 0.5319751	total: 2m 52s	remaining: 6m 2s
323:	learn: 0.5310569	total: 2m 53s	remaining: 6m 1s
324:	learn: 0.5299286	total: 2m 53s	remaining: 6m 1s
325:	learn: 0.5293951	total: 2m 54s	remaining: 6m
326:	learn: 0.5288264	total: 2m 54s	remaining: 6m
327:	learn: 0.5281152	total: 2m 55s	remaining: 5m 59s
328:	learn: 0.5271851	total: 2m 56s	remaining: 5m 5

464:	learn: 0.4358274	total: 4m 17s	remaining: 4m 56s
465:	learn: 0.4354851	total: 4m 18s	remaining: 4m 55s
466:	learn: 0.4352060	total: 4m 18s	remaining: 4m 55s
467:	learn: 0.4345955	total: 4m 19s	remaining: 4m 54s
468:	learn: 0.4342209	total: 4m 20s	remaining: 4m 54s
469:	learn: 0.4338190	total: 4m 20s	remaining: 4m 53s
470:	learn: 0.4336957	total: 4m 21s	remaining: 4m 53s
471:	learn: 0.4333856	total: 4m 21s	remaining: 4m 52s
472:	learn: 0.4331261	total: 4m 22s	remaining: 4m 52s
473:	learn: 0.4323714	total: 4m 22s	remaining: 4m 51s
474:	learn: 0.4319829	total: 4m 23s	remaining: 4m 51s
475:	learn: 0.4315599	total: 4m 24s	remaining: 4m 50s
476:	learn: 0.4313235	total: 4m 24s	remaining: 4m 50s
477:	learn: 0.4310225	total: 4m 25s	remaining: 4m 49s
478:	learn: 0.4304846	total: 4m 26s	remaining: 4m 49s
479:	learn: 0.4294675	total: 4m 26s	remaining: 4m 48s
480:	learn: 0.4289081	total: 4m 27s	remaining: 4m 48s
481:	learn: 0.4283091	total: 4m 27s	remaining: 4m 47s
482:	learn: 0.4279896	total:

617:	learn: 0.3726050	total: 5m 47s	remaining: 3m 34s
618:	learn: 0.3719202	total: 5m 48s	remaining: 3m 34s
619:	learn: 0.3716104	total: 5m 48s	remaining: 3m 33s
620:	learn: 0.3712961	total: 5m 49s	remaining: 3m 33s
621:	learn: 0.3706646	total: 5m 50s	remaining: 3m 32s
622:	learn: 0.3700676	total: 5m 50s	remaining: 3m 32s
623:	learn: 0.3697020	total: 5m 51s	remaining: 3m 31s
624:	learn: 0.3691960	total: 5m 51s	remaining: 3m 31s
625:	learn: 0.3687558	total: 5m 52s	remaining: 3m 30s
626:	learn: 0.3685789	total: 5m 52s	remaining: 3m 29s
627:	learn: 0.3682729	total: 5m 53s	remaining: 3m 29s
628:	learn: 0.3676155	total: 5m 54s	remaining: 3m 28s
629:	learn: 0.3671339	total: 5m 54s	remaining: 3m 28s
630:	learn: 0.3663121	total: 5m 55s	remaining: 3m 27s
631:	learn: 0.3655227	total: 5m 56s	remaining: 3m 27s
632:	learn: 0.3654016	total: 5m 56s	remaining: 3m 26s
633:	learn: 0.3652443	total: 5m 57s	remaining: 3m 26s
634:	learn: 0.3650194	total: 5m 57s	remaining: 3m 25s
635:	learn: 0.3647163	total:

770:	learn: 0.3202824	total: 7m 20s	remaining: 2m 10s
771:	learn: 0.3202427	total: 7m 20s	remaining: 2m 10s
772:	learn: 0.3200519	total: 7m 21s	remaining: 2m 9s
773:	learn: 0.3199565	total: 7m 21s	remaining: 2m 8s
774:	learn: 0.3197208	total: 7m 22s	remaining: 2m 8s
775:	learn: 0.3195923	total: 7m 22s	remaining: 2m 7s
776:	learn: 0.3194755	total: 7m 23s	remaining: 2m 7s
777:	learn: 0.3192988	total: 7m 24s	remaining: 2m 6s
778:	learn: 0.3189574	total: 7m 24s	remaining: 2m 6s
779:	learn: 0.3186759	total: 7m 25s	remaining: 2m 5s
780:	learn: 0.3184155	total: 7m 26s	remaining: 2m 5s
781:	learn: 0.3181751	total: 7m 26s	remaining: 2m 4s
782:	learn: 0.3177337	total: 7m 27s	remaining: 2m 3s
783:	learn: 0.3174749	total: 7m 27s	remaining: 2m 3s
784:	learn: 0.3170528	total: 7m 28s	remaining: 2m 2s
785:	learn: 0.3165926	total: 7m 29s	remaining: 2m 2s
786:	learn: 0.3164340	total: 7m 29s	remaining: 2m 1s
787:	learn: 0.3163008	total: 7m 30s	remaining: 2m 1s
788:	learn: 0.3160807	total: 7m 30s	remainin

924:	learn: 0.2830292	total: 8m 49s	remaining: 42.9s
925:	learn: 0.2828637	total: 8m 50s	remaining: 42.4s
926:	learn: 0.2826502	total: 8m 50s	remaining: 41.8s
927:	learn: 0.2824940	total: 8m 51s	remaining: 41.2s
928:	learn: 0.2823985	total: 8m 52s	remaining: 40.7s
929:	learn: 0.2819784	total: 8m 52s	remaining: 40.1s
930:	learn: 0.2818250	total: 8m 53s	remaining: 39.5s
931:	learn: 0.2816730	total: 8m 53s	remaining: 39s
932:	learn: 0.2815076	total: 8m 54s	remaining: 38.4s
933:	learn: 0.2813406	total: 8m 55s	remaining: 37.8s
934:	learn: 0.2811941	total: 8m 55s	remaining: 37.2s
935:	learn: 0.2811681	total: 8m 56s	remaining: 36.7s
936:	learn: 0.2809870	total: 8m 56s	remaining: 36.1s
937:	learn: 0.2805261	total: 8m 57s	remaining: 35.5s
938:	learn: 0.2801235	total: 8m 57s	remaining: 34.9s
939:	learn: 0.2799016	total: 8m 58s	remaining: 34.4s
940:	learn: 0.2797826	total: 8m 58s	remaining: 33.8s
941:	learn: 0.2796530	total: 8m 59s	remaining: 33.2s
942:	learn: 0.2795258	total: 8m 59s	remaining: 3

In [126]:
import matplotlib.pyplot as plt
pd.DataFrame(zip(clf.feature_names_,clf.feature_importances_)).sort_values(by=1,ascending=False)


,0,1
184,184,4.262323
35,35,3.685491
14,14,3.535345
183,183,3.348064
18,18,3.286546
...,...,...
89,89,0.003334
230,230,0.001159
236,236,0.000000
258,258,0.000000


In [97]:
df3.to_csv('final.csv')

In [77]:
roc_auc_score(y_test,clf.predict_proba(X_test),multi_class='ovo')
print(classification_report(y_test,clf.predict(X_test)))
f1_score(y_test,clf.predict(X_test),average='micro')

              precision    recall  f1-score   support

         0.0       0.83      0.83      0.83      1379
         1.0       0.91      0.89      0.90      1979
         2.0       0.90      0.93      0.92      1911
         3.0       0.82      0.88      0.85      1611
         4.0       0.89      0.89      0.89      1713
         5.0       0.90      0.97      0.93      1727
         6.0       0.87      0.78      0.82      1770
         7.0       0.87      0.85      0.86      1562
         8.0       0.83      0.82      0.82      1249
         9.0       0.90      0.86      0.88      1638

    accuracy                           0.87     16539
   macro avg       0.87      0.87      0.87     16539
weighted avg       0.87      0.87      0.87     16539



0.8744785053509886

In [48]:
df2 = pd.DataFrame(rt[1:])
df2

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.930780,0.046606,0.000182,0.000000,2196.936343,0.000000,2530.983967,0.000000,4360.473633,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.344606,0.323810,0.084774,0.076660,1105.445041,631.949733,1356.294071,461.144252,2004.327111,1492.209987,...,0.000716,0.000693,0.000682,0.000670,0.000659,0.000655,0.000646,0.000641,0.000637,0.0
2,0.477368,0.273781,0.307928,0.048397,1244.101522,625.729326,1731.113100,609.054975,2650.820125,1706.438651,...,0.098661,0.077175,0.035065,0.024645,0.047935,0.029966,0.011757,0.005383,0.000808,0.0
3,0.485398,0.273511,0.318516,0.048662,1239.821770,631.726391,1726.209890,601.361678,2629.949482,1713.737147,...,0.107550,0.087843,0.041335,0.029646,0.054276,0.032650,0.010951,0.005443,0.000776,0.0
4,0.471938,0.270860,0.300841,0.052760,1248.214680,632.930164,1707.341627,592.947430,2643.697604,1687.612777,...,0.105501,0.078920,0.035369,0.025382,0.051151,0.029620,0.010899,0.005154,0.000870,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,0.484134,0.279775,0.226370,0.046792,2163.291277,1032.497846,2386.891183,549.922434,4630.963792,2160.571838,...,0.092213,0.073355,0.082875,0.116733,0.061459,0.046441,0.049033,0.020216,0.001387,9.0
22990,0.477421,0.276692,0.234049,0.043795,1711.707338,878.638994,2157.712807,612.977281,3808.064153,2121.233578,...,0.068765,0.066640,0.063687,0.058746,0.027141,0.022732,0.041965,0.026203,0.000942,9.0
22991,0.477290,0.281485,0.230659,0.038390,2102.181044,942.641862,2341.503765,544.800525,4645.954214,1955.207471,...,0.104100,0.072463,0.080845,0.104704,0.048883,0.030862,0.020895,0.012920,0.000884,9.0
22992,0.489672,0.275708,0.218926,0.040345,1906.249839,930.098140,2276.780982,563.024192,4202.618690,2016.781544,...,0.106424,0.116938,0.131141,0.110704,0.043094,0.035499,0.043017,0.027297,0.001773,9.0


In [11]:
import warnings
warnings.filterwarnings('ignore')
yandex_genre = dict(list(zip(genres,[0 for _ in range(10)])))
for k,genre in enumerate(genresgt):
    ans = np.zeros((311,))
    songs = os.listdir(path2+f'/{genre}')
    print(genre)
    for j,s in (enumerate(songs)):
        if j == 35:
            break
        if s == '.DS_Store':
            continue
        try:
            dur = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration']
            durts = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration_ts']
        except:
            pass
        datak, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        count = math.floor(float(dur)/3)
        i = 0
        print(count)
        while i <= count:
            data = datak[i*66149:66149*(i+1)]
            mel = lr.feature.melspectrogram(y=data,sr=sr)
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            melmean = np.mean(mel,axis=1)
            melvar = np.std(mel,axis=1)
            prom = list()
            for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar,melmean,melvar]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
            i+=1
    ans = np.hstack((ans,np.array([k for i in range(ans.shape[0])]).reshape(-1,1)))
    yandex_genre[genre] = ans

pop
54
54
57
62
45
47
70
50
58
63
49
51
55
20
53
47
54
62
64
40
79
60
57
53
52
67
48
81
53
64
46
79
51
64
37
metal
99
76
89
80
90
73
71
54
85
94
34
99
94
88
68
84
105
90
83
79
76
93
67
76
72
57
82
76
43
57
83
78
103
93
35
disco
83
78
66
70
98
75
80
83
72
171
71
95
72
75
72
70
80
80
74
131
59
64
66
104
67
102
67
117
116
59
61
70
81
113
91
blues
83
56
115
59
57
73
61
59
60
54
65
60
71
71
48
61
80
63
60
55
90
67
99
57
60
207
77
75
71
67
66
56
149
60
55
reggae
41
49
69
75
75
81
83
105
69
47
76
64
51
71
60
89
84
80
89
58
59
60
66
103
72
66
69
87
74
86
50
70
80
58
91
classical
70
67
54
56
90
44
68
122
113
115
83
45
37
60
145
85
136
71
44
37
96
95
47
122
47
60
60
63
100
33
70
9
69
85
54
rock
67
81
71
51
65
69
77
79
51
71
69
75
47
89
86
74
100
74
53
58
63
66
63
72
89
52
79
61
77
53
56
64
53
89
85
hip hop
48
36
61
50
79
55
43
43
84
71
74
70
57
45
60
50
58
45
81
64
47
65
83
62
46
57
84
49
47
55
54
76
40
54
45
country
118
78
51
49
70
41
84
82
88
78
67
67
56
29
73
41
60
69
65
83
jazz
76
62
137
69


In [32]:
rt = np.zeros((312,))
for i in yandex_genre:
    if type(yandex_genre[i]) != int:
        yandex_genre[i] = yandex_genre[i][:-1]
        rt = np.vstack((rt,yandex_genre[i]))
df = pd.DataFrame(rt[1:])

df

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.930780,0.046606,0.000182,0.000000,2196.936343,0.000000,2530.983967,0.000000,4360.473633,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.344606,0.323810,0.084774,0.076660,1105.445041,631.949733,1356.294071,461.144252,2004.327111,1492.209987,...,0.000716,0.000693,0.000682,0.000670,0.000659,0.000655,0.000646,0.000641,0.000637,0.0
2,0.477368,0.273781,0.307928,0.048397,1244.101522,625.729326,1731.113100,609.054975,2650.820125,1706.438651,...,0.098661,0.077175,0.035065,0.024645,0.047935,0.029966,0.011757,0.005383,0.000808,0.0
3,0.485398,0.273511,0.318516,0.048662,1239.821770,631.726391,1726.209890,601.361678,2629.949482,1713.737147,...,0.107550,0.087843,0.041335,0.029646,0.054276,0.032650,0.010951,0.005443,0.000776,0.0
4,0.471938,0.270860,0.300841,0.052760,1248.214680,632.930164,1707.341627,592.947430,2643.697604,1687.612777,...,0.105501,0.078920,0.035369,0.025382,0.051151,0.029620,0.010899,0.005154,0.000870,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,0.484134,0.279775,0.226370,0.046792,2163.291277,1032.497846,2386.891183,549.922434,4630.963792,2160.571838,...,0.092213,0.073355,0.082875,0.116733,0.061459,0.046441,0.049033,0.020216,0.001387,9.0
22990,0.477421,0.276692,0.234049,0.043795,1711.707338,878.638994,2157.712807,612.977281,3808.064153,2121.233578,...,0.068765,0.066640,0.063687,0.058746,0.027141,0.022732,0.041965,0.026203,0.000942,9.0
22991,0.477290,0.281485,0.230659,0.038390,2102.181044,942.641862,2341.503765,544.800525,4645.954214,1955.207471,...,0.104100,0.072463,0.080845,0.104704,0.048883,0.030862,0.020895,0.012920,0.000884,9.0
22992,0.489672,0.275708,0.218926,0.040345,1906.249839,930.098140,2276.780982,563.024192,4202.618690,2016.781544,...,0.106424,0.116938,0.131141,0.110704,0.043094,0.035499,0.043017,0.027297,0.001773,9.0


In [3]:
def predict_genre(song:str, model, true_class:int) -> np.array:
    ans = np.zeros((311,))
    try:
        dur = pydub.utils.mediainfo(song)['duration']
        durts = pydub.utils.mediainfo(song)['duration_ts']
    except:
        raise ValueError('no song')
    datak, sr = lr.load(song,  duration = math.floor(float(dur)))
    count = math.floor(float(dur)/3)
    i = 0
    while i <= count:
        data = datak[i*66149:66149*(i+1)]
        mel = lr.feature.melspectrogram(y=data,sr=sr)
        mfcc = lr.feature.mfcc(y=data,sr=sr)
        stft = lr.feature.chroma_stft(y=data, sr=sr)
        rms = lr.feature.rms(y=data)
        centroid = lr.feature.spectral_centroid(y=data, sr=sr)
        bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
        rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
        zero = lr.feature.zero_crossing_rate(y=data)
        harm = lr.effects.harmonic(y=data)
        temp = lr.feature.tempo(y=data)
        mfccmean = np.mean(mfcc,axis=1)
        mfccvar = np.std(mfcc,axis=1)
        stftmean = np.mean(stft)
        stftvar = np.std(stft)
        rmsmean = np.mean(rms)
        rmsvar = np.std(rms)
        centroidmean = np.mean(centroid)
        centroidvar = np.std(centroid)
        bandwithmean = np.mean(bandwith)
        bandwithvar = np.std(bandwith)
        rolloffmean = np.mean(rolloff)
        rolloffvar = np.std(rolloff)
        zeromean = np.mean(zero)
        zerovar = np.std(zero)
        harmmean = np.mean(harm)
        harmvar = np.std(harm)
        melmean = np.mean(mel,axis=1)
        melvar = np.std(mel,axis=1)
        prom = list()
        for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar,melmean,melvar]:
            if type(feat) == np.ndarray:
                prom += feat.tolist()
            else:
                prom.append(feat)
        prom = np.array(prom)
        ans = np.vstack((prom,ans))
        i+=1
    predict = clf.predict(ans[:-1])    
    our_class = pd.Series(predict.reshape(1,-1)[0]).value_counts().sort_values(ascending=False)
    if true_class == our_class.index[0]:
        return [True,our_class.values[0]/(our_class.values[0]+our_class.values[1])]
    else:
        return [False,our_class.values[0]/(our_class.values[0]+our_class.values[1])]